In [28]:
import os
from dotenv import load_dotenv , find_dotenv

_ =load_dotenv(find_dotenv())



Prompts and prompt templates

In [29]:
# for completion model
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
tem = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)
chatm = ChatGroq(
    model="llama3-70b-8192"
)

pro=tem.format(
    adjective = "curious",
    topic="The kennedy family"
)

print(chatm.invoke(pro).content)



Here's a curious story about the Kennedy family:

**The Mysterious Disappearance of Rosemary Kennedy**

Rosemary Kennedy, the third child and first daughter of Joseph P. Kennedy Sr. and Rose Fitzgerald Kennedy, was born in 1918. Growing up, Rosemary was known to be a sweet and gentle soul, but she struggled with intellectual disabilities and learning difficulties. Despite her challenges, Rosemary was a beloved member of the Kennedy family and was particularly close to her siblings, including John F. Kennedy, who would later become the President of the United States.

In 1941, when Rosemary was 23 years old, her parents, desperate to "cure" her of her disabilities, made a fateful decision. They arranged for Rosemary to undergo a prefrontal lobotomy, a then-experimental procedure that involved severing connections in the brain in an attempt to alleviate mental health issues. The operation was performed by two inexperienced doctors, who carried out the procedure without the family's full 

In [30]:
# for chat completion model
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
tem = ChatPromptTemplate.from_messages(
    [
        ("system","You are an {profession} expert on {topic}."),
        ("human","Hello, Mr.{profession}, can you please answer a question?"),
        ("ai","Sure!"),
        ("human","{user_input}"),
    ]
)
chatm = ChatGroq(
    model="llama3-70b-8192"
)

pro=tem.format_messages(
    profession = "historian",
    topic="The kennedy family",
    user_input="How many grandchildren has Joseph P. Kennedy?"
)

print(chatm.invoke(pro).content)



Joseph P. Kennedy, the patriarch of the Kennedy family, had a total of 29 grandchildren. He had nine children with his wife Rose Fitzgerald Kennedy, and those children went on to have numerous offspring of their own. In fact, the Kennedy family is known for its large and sprawling family tree, with many descendants of Joseph and Rose still active in politics, business, and public life today.


Few Shot Prompting

In [31]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

exampl=[
    {"input":"hi","output":"hola"},
    {"input":"bye","output":"adios"}
]

proo = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few = FewShotChatMessagePromptTemplate(
    example_prompt=proo,
    examples=exampl,

)

final = ChatPromptTemplate.from_messages(
    [
        ("system","You are an english-spanish translator"),
        few,
        ("human","{input}"),
    ]
)



Chains

In [32]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

exampl=[
    {"input":"hi","output":"hola"},
    {"input":"bye","output":"adios"}
]

proo = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few = FewShotChatMessagePromptTemplate(
    example_prompt=proo,
    examples=exampl,

)

final = ChatPromptTemplate.from_messages(
    [
        ("system","You are an english-spanish translator"),
        few,
        ("human","{input}"),
    ]
)

chain = final | chatm

chain.invoke({"input":"How are you?"}).content

'¿Cómo estás?'

In [33]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_p= PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question : {question}"
)
json_pa = SimpleJsonOutputParser()
json_ch = json_p | chatm | json_pa
json_ch.invoke({"question":"what is the biggest country"})

{'answer': 'Russia'}

Optionally , you can use pydantic to define a custom output format

In [34]:
from langchain_core.pydantic_v1 import BaseModel,Field

In [35]:
#Define a pydantic object with the desired output format.

class joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline : str = Field(description="answer to resolve the joke")

In [36]:
#Define the parser refering the pydantic object
parser = SimpleJsonOutputParser(pydantic_object=joke)

#Add the parset format instructions in the prompt definition.
prompt = PromptTemplate(
    template = "Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()},
)

#create a chain with the prompt and the parser
chain = prompt | chatm | parser

chain.invoke({"query":"Tell me a joke."})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}